## Scraping the Data of Bangladeshi Restaurants
In this project we'll use Google Places API and Python programming language to scrape the data of restaurants all over Bangladesh.

In [11]:
# Import libraries
import pandas as pd
import json
import requests
import time

In [12]:
# Define a class
class PlacesInfo(object):
    def __init__(self, apiKey):
        super(PlacesInfo, self).__init__()
        self.apiKey = apiKey

    # Define a function to get places
    def find_places_by_location(self, location, radius, types):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        places = []
        params = {
            "location": location,
            "radius": radius,
            "types": types,
            "key": self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        results =  json.loads(res.content)
        places.extend(results["results"])
        time.sleep(2)
        while "next_page_token" in results:
            params["pagetoken"] = results["next_page_token"],
            res = requests.get(endpoint_url, params = params)
            results = json.loads(res.content)
            places.extend(results["results"])
            time.sleep(2)
        return places

    # Define a function to get details of places
    def find_place_details(self, place_id, fields):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/details/json"
        params = {
            "placeid": place_id,
            "fields": ",".join(fields),
            "key": self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        place_details =  json.loads(res.content)
        return place_details

In [13]:
# Specify the API Key (keeping empty for the time being, but have to put a valid key)
API_KEY = ""

# Instantiate the class
api = PlacesInfo(API_KEY)

# Call the function to get places
# Pass the location (Latitude, Longitude) and approximate radius of Bangladesh along with restaurant as type
places = api.find_places_by_location("23.6850,90.3563", "410000", "restaurant")

# Specify the fields
fields = ["name", "geometry", "rating", "user_ratings_total", "price_level"]

# Create an empty DataFrame to contain the output
df = pd.DataFrame(columns=["Name", "Latitude", "Longitude", "Rating", "Number of reviews", "Affluence"])

# Iterate through the places and get details of each place
for place in places:
    details = api.find_place_details(place["place_id"], fields)
    try:
        name = details["result"]["name"]
    except KeyError:
        name = ""

    try:
        latitude = details["result"]["geometry"]["location"]["lat"]
    except KeyError:
        latitude = ""
    
    try:
        longitude = details["result"]["geometry"]["location"]["lng"]
    except KeyError:
        longitude = ""

    try:
        rating = details["result"]["rating"]
    except KeyError:
        rating = ""

    try:
        num_reviews = details["result"]["user_ratings_total"]
    except KeyError:
        num_reviews = ""

    try:
        affluence = details["result"]["price_level"]
    except KeyError:
        affluence = ""
    
    print("===================PLACE DETAILS===================")
    print("Name:", name)
    print("Latitude:", latitude)
    print("Longitude:", longitude)
    print("Rating", rating)
    print("Number of reviews:", num_reviews)
    print("Affluence:", affluence)
    
    # Create a temporary DataFrame from the field values and concat it with the main DataFrame
    temp_list = [[name, latitude, longitude, rating, num_reviews, affluence]]
    temp_df = pd.DataFrame(temp_list, columns=["Name", "Latitude", "Longitude", "Rating", "Number of reviews", "Affluence"])
    df = pd.concat([df, temp_df])

# Export the output to a CSV file
df.to_csv("./restaurant-data.csv", index=False)
    